In [1]:
import pickle

In [2]:
with open('./site_dic.pkl', 'rb') as f:
    site_dict = pickle.load(f)

In [3]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

from time import time

In [4]:
sites = ['site%s' % i for i in range(1, 11)]

In [5]:
df_train = pd.read_csv('train_sessions.csv', dtype={s: 'str' for s in sites})

In [82]:
df_train['nulls'] = df_train.isnull().sum(axis=1)

In [6]:
for s in sites:
    df_train[s].fillna('', inplace=1)

In [7]:
df = df_train

In [8]:
df['sites'] = df.site1 + ' ' + df.site2 + ' ' + df.site3 + ' ' + df.site4 + ' ' + df.site5 + ' ' + \
              df.site6 + ' ' + df.site7 + ' ' + df.site8 + ' ' + df.site9 + ' ' + df.site10  

In [42]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.model_selection import train_test_split

In [97]:
cv_cnt = CountVectorizer(token_pattern='\S+', dtype=np.uint8, binary=False)
X_cnt = cv_cnt.fit_transform(df_train.sites)

cv = TfidfVectorizer(token_pattern='\S+', min_df=10, max_df=0.5, ngram_range=(1, 3))
X_ohe = cv.fit_transform(df_train.sites)

X_ohe.shape

(253561, 39961)

In [102]:
df_train['max_cnt'] = X_cnt.max(axis=1).toarray()[:, 0]

In [102]:
df_train['max_cnt'] = 

In [103]:
X_num = df_train[['nulls', 'max_cnt']].values

In [83]:
import scipy.sparse as sp

In [104]:
X = sp.hstack([X_num, X_ohe], format='csr')

In [105]:
y = df_train.target.values

In [106]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

In [107]:
for C in [0.01, 0.1, 0.5, 1]:
    t0 = time()

    #svm = LinearSVC(penalty='l1', dual=False, C=C, random_state=1)
    svm = LogisticRegression(penalty='l1', dual=False, C=C, random_state=1)
    svm.fit(X_train, y_train)

    y_pred = svm.decision_function(X_val)
    auc = roc_auc_score(y_val, y_pred)

    print('C=%s, took %.3fs, auc=%.3f' % (C, time() - t0, auc))

C=0.01, took 0.957s, auc=0.645
C=0.1, took 1.178s, auc=0.836
C=0.5, took 1.500s, auc=0.932
C=1, took 1.710s, auc=0.945
C=5, took 4.924s, auc=0.952


In [94]:
for C in [0.01, 0.1, 0.5, 1, 5]:
    t0 = time()

    #svm = LinearSVC(penalty='l1', dual=False, C=C, random_state=1)
    svm = LogisticRegression(penalty='l1', dual=False, C=C, random_state=1)
    svm.fit(X_train, y_train)

    y_pred = svm.decision_function(X_val)
    auc = roc_auc_score(y_val, y_pred)

    print('C=%s, took %.3fs, auc=%.3f' % (C, time() - t0, auc))

C=0.01, took 0.477s, auc=0.651
C=0.1, took 0.769s, auc=0.848
C=0.5, took 0.993s, auc=0.933
C=1, took 1.240s, auc=0.946
C=5, took 2.835s, auc=0.952


In [41]:
for C in [0.01, 0.1, 0.5, 1]:
    t0 = time()

    svm = LinearSVC(penalty='l1', dual=False, C=C, random_state=1)
    #svm = LogisticRegression(penalty='l1', dual=False, C=C, random_state=1)
    svm.fit(X_train, y_train)

    y_pred = svm.decision_function(X_val)
    auc = roc_auc_score(y_val, y_pred)

    print('C=%s, took %.3fs, auc=%.3f' % (C, time() - t0, auc))

C=0.01, took 0.861s, auc=0.890
C=0.1, took 4.942s, auc=0.933
C=0.5, took 9.134s, auc=0.916
C=1, took 9.839s, auc=0.891


In [64]:
#C = 0.1
#svm = LinearSVC(penalty='l1', dual=False, C=C, random_state=1)
C = 1
svm =LogisticRegression(penalty='l1', dual=False, C=C, random_state=1)
svm.fit(X, y)

LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

logreg, tf-idf 1-3 ngrams

- C=1, 0.91868, cv = auc=0.945
- C=5, 0.91512, cv = auc=0.952

In [65]:
df_test = pd.read_csv('test_sessions.csv', dtype={s: 'str' for s in sites})

for s in sites:
    df_test[s].fillna('', inplace=1)

In [66]:
df = df_test
df['sites'] = df.site1 + ' ' + df.site2 + ' ' + df.site3 + ' ' + df.site4 + ' ' + df.site5 + ' ' + \
              df.site6 + ' ' + df.site7 + ' ' + df.site8 + ' ' + df.site9 + ' ' + df.site10  

In [67]:
X_test = cv.transform(df_test.sites)

In [68]:
pred = svm.decision_function(X_test)

In [69]:
df_res = pd.DataFrame()
df_res['session_id'] = df_test.session_id
df_res['target'] = pred

In [70]:
df_res.to_csv('benchmark6.csv', index=False)

In [71]:
!gzip -f benchmark6.csv

- 1 -> LB: 89
- 0.1 -> 0.91416